In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
%tensorflow_version 1.x
from keras.models import load_model
from __future__ import print_function
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras import applications
import cv2 
from os import listdir
import numpy as np
import matplotlib. pyplot as plt
from keras import layers
from keras.models import Sequential
from keras.layers import Input
from keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten, AveragePooling2D,GlobalAveragePooling2D,Activation,BatchNormalization
from keras.models import Model
from keras.applications import DenseNet121
from keras.applications.inception_v3 import InceptionV3
from keras.optimizers import Adam, Adamax
from keras.callbacks import LearningRateScheduler
from keras import optimizers
#Augmentation
from numpy import expand_dims
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import ImageDataGenerator
from matplotlib import pyplot
from sklearn.utils import shuffle
from sklearn.utils import class_weight
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, ReduceLROnPlateau, TensorBoard, EarlyStopping
from sklearn.model_selection import KFold,StratifiedKFold
from sklearn.model_selection import train_test_split
import urllib,math

TensorFlow 1.x selected.


Using TensorFlow backend.


In [0]:
kaggle_pneumonia = np.load('/content/drive/My Drive/Corona_Image/Pavel/Data/v3/kaggle_pneumonia_images.npy')
kaggle_normal = np.load('/content/drive/My Drive/Corona_Image/Pavel/Data/v3/kaggle_normal_images.npy')
ieee_corona=np.load('/content/drive/My Drive/Corona_Image/Pavel/Data/v3/ieee8023_covid_images.npy')
ieee_pneumonia=np.load('/content/drive/My Drive/Corona_Image/Pavel/Data/v3/ieee8023_pneumonia_images.npy')
figure1 = np.load('/content/drive/My Drive/Corona_Image/Pavel/Data/v2/figure1_covid.npy')
figure1_mask = np.load('/content/drive/My Drive/Corona_Image/Pavel/Data/v2/figure1_covid_mask.npy')
figure1_corona = figure1[figure1_mask]

In [0]:
covid_folds = np.array([[186, 168, 167, 59, 57, 158, 157, 93, 85, 56, 55, 32, 24],
 [178, 169, 164, 67, 51, 159, 156, 94, 84, 60, 54, 33, 23],
 [173, 183, 163, 68, 37, 161, 155, 99, 83, 61, 53, 34, 22],
 [142, 185, 150, 69, 36, 162, 154, 118, 82, 62, 52, 35, 21],
 [19, 15, 144, 70, 6, 165, 153, 137, 81, 63, 50, 39, 20],
 [13, 17, 143, 73, 4, 166, 152, 138, 80, 64, 49, 40, 18],
 [2, 58, 139, 86, 184, 174, 151, 140, 79, 65, 48, 41, 16],
 [187, 71, 115, 95, 182, 175, 149, 141, 78, 72, 47, 42, 14],
 [179, 116, 114, 98, 181, 176, 148, 145, 77, 74, 46, 43, 12],
 [132, 117, 113, 112, 180, 177, 147, 146, 76, 75, 45, 44, 11]]
)
pnemonia_folds = np.array([[31],
 [171],
 [87, 29],
 [10, 30],
 [3, 88],
 [92, 89],
 [9, 90],
 [7, 100],
 [172, 110],
 [170, 111, 8]]
)

In [0]:
covid_patient_id=np.load('/content/drive/My Drive/Corona_Image/Pavel/Data/v3/ieee8023_covid_images_patient_ids.npy')
pneumonia_patient_id=np.load('/content/drive/My Drive/Corona_Image/Pavel/Data/v3/ieee8023_pneumonia_images_patient_ids.npy')

In [0]:
# fold_no=10
# mask = np.isin(covid_patient_id, covid_folds[fold_no-1])
# test = ieee_corona[mask]
# mask2 = np.isin(covid_patient_id, covid_folds[fold_no%10])
# validation = ieee_corona[mask2]
# arr = np.concatenate((covid_folds[fold_no-1], covid_folds[fold_no%10]), axis=0)
# mask3 = np.isin(covid_patient_id, arr, invert=True)
# train = ieee_corona[mask3]
# print(train.shape,validation.shape,test.shape)

In [0]:
# num_folds = 10
# kfold = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=1)
# label = np.empty(figure1_corona.shape[0],dtype='int32')
# label.fill(2)
# fold_no=1
# for train,test in kfold.split(figure1_corona,label): 
#    np.save(f'/content/drive/My Drive/Corona_Image/Final_data/figure1/train{fold_no}.npy',train)
#    np.save(f'/content/drive/My Drive/Corona_Image/Final_data/figure1/test{fold_no}.npy',test)
#    fold_no=fold_no+1

In [0]:
def augment(images,num):
  aug = []
  datagens = []
  datagens.append(ImageDataGenerator(width_shift_range=0.15))
  datagens.append(ImageDataGenerator(height_shift_range=0.15))
  datagens.append(ImageDataGenerator(horizontal_flip=True))
  datagens.append(ImageDataGenerator(rotation_range=45))
  datagens.append(ImageDataGenerator(brightness_range=[0.6,1.0]))
  datagens.append(ImageDataGenerator(zoom_range=0.15))
  for data in images:
    samples = expand_dims(data, 0)
    for datagen in datagens:
      it = datagen.flow(samples, batch_size=1)
      for i in range(num):
        batch = it.next()
        image = batch[0].astype('uint8')
        aug.append(image)
  return aug

In [0]:
def load_train(fold_no):
    k_pneumonia = np.load(f'/content/drive/My Drive/Corona_Image/Final_data/kaggle_pneumonia/train{fold_no}.npy')
    nor = np.load(f'/content/drive/My Drive/Corona_Image/Final_data/normal/train{fold_no}.npy')
    fig = np.load(f'/content/drive/My Drive/Corona_Image/Final_data/figure1/train{fold_no}.npy')

    mask = np.isin(covid_patient_id, covid_folds[fold_no%10])
    val_data = ieee_corona[mask]
    arr = np.concatenate((covid_folds[fold_no-1], covid_folds[fold_no%10]), axis=0)
    mask = np.isin(covid_patient_id, arr, invert=True)
    train_data = ieee_corona[mask]
    
    figure1 = figure1_corona[fig]
    train_data2, val_data2 = train_test_split(figure1, test_size=0.1, random_state=42)
    train_data = np.concatenate((train_data,train_data2), axis=0)
    aug = np.array(augment(train_data,9))
    train_data = np.concatenate((train_data,aug), axis=0)
    train_label = np.empty(train_data.shape[0],dtype='int32')
    train_label.fill(2)
    val_data = np.concatenate((val_data,val_data2), axis=0)
    aug = np.array(augment(val_data,9))
    val_data = np.concatenate((val_data,aug), axis=0)
    val_label = np.empty(val_data.shape[0],dtype='int32')
    val_label.fill(2)

    mask = np.isin(pneumonia_patient_id, pnemonia_folds[fold_no%10])
    val_data2 = ieee_pneumonia[mask]
    val_data = np.concatenate((val_data,val_data2), axis=0)
    l = np.empty(val_data2.shape[0],dtype='int32')
    l.fill(1)
    val_label = np.concatenate((val_label, l), axis=0)
    arr = np.concatenate((pnemonia_folds[fold_no-1], pnemonia_folds[fold_no%10]), axis=0)
    mask = np.isin(pneumonia_patient_id, arr, invert=True)
    train_data2 = ieee_pneumonia[mask]
    train_data = np.concatenate((train_data,train_data2), axis=0)
    l = np.empty(train_data2.shape[0],dtype='int32')
    l.fill(1)
    train_label = np.concatenate((train_label, l), axis=0)

    pneumonia = kaggle_pneumonia[k_pneumonia]
    train_data2, val_data2 = train_test_split(pneumonia, test_size=0.1, random_state=42)
    train_data = np.concatenate((train_data,train_data2), axis=0)
    l = np.empty(train_data2.shape[0],dtype='int32')
    l.fill(1)
    train_label = np.concatenate((train_label, l), axis=0)
    val_data = np.concatenate((val_data,val_data2), axis=0)
    l = np.empty(val_data2.shape[0],dtype='int32')
    l.fill(1)
    val_label = np.concatenate((val_label, l), axis=0)

    normal = kaggle_normal[nor]
    train_data2, val_data2 = train_test_split(normal, test_size=0.1, random_state=42)
    train_data = np.concatenate((train_data,train_data2), axis=0)
    l = np.empty(train_data2.shape[0],dtype='int32')
    l.fill(0)
    train_label = np.concatenate((train_label, l), axis=0)
    val_data = np.concatenate((val_data,val_data2), axis=0)
    l = np.empty(val_data2.shape[0],dtype='int32')
    l.fill(0)
    val_label = np.concatenate((val_label, l), axis=0)
    
    train_data,train_label = shuffle(train_data,train_label)
    val_data,val_label = shuffle(val_data,val_label)
    return train_data, val_data, train_label, val_label

In [0]:
def load_test(fold_no):
    k_pneumonia = np.load(f'/content/drive/My Drive/Corona_Image/Final_data/kaggle_pneumonia/test{fold_no}.npy')
    nor = np.load(f'/content/drive/My Drive/Corona_Image/Final_data/normal/test{fold_no}.npy')
    fig = np.load(f'/content/drive/My Drive/Corona_Image/Final_data/figure1/test{fold_no}.npy')

    mask = np.isin(covid_patient_id, covid_folds[fold_no-1])
    data = ieee_corona[mask]
    figure1 = figure1_corona[fig]
    data = np.concatenate((data,figure1), axis=0)
    label = np.empty(data.shape[0],dtype='int32')
    label.fill(2)

    mask = np.isin(pneumonia_patient_id, pnemonia_folds[fold_no-1])
    pneumonia = ieee_pneumonia[mask]
    data = np.concatenate((data, pneumonia), axis=0)
    l = np.empty(pneumonia.shape[0],dtype='int32')
    l.fill(1)
    label = np.concatenate((label, l), axis=0)

    pneumonia = kaggle_pneumonia[k_pneumonia]
    data = np.concatenate((data, pneumonia), axis=0)
    l = np.empty(pneumonia.shape[0],dtype='int32')
    l.fill(1)
    label = np.concatenate((label, l), axis=0)

    normal = kaggle_normal[nor]
    data = np.concatenate((data, normal), axis=0)
    l = np.empty(normal.shape[0],dtype='int32')
    l.fill(0)
    label = np.concatenate((label, l), axis=0)
    
    data,label=shuffle(data,label)

    return data,label

In [0]:
def define_mode(l,batch):
    # f.write(str(l)+","+str(batch)+","+str(drop)+",")
    # Define per-fold score containers
    acc_per_fold = []
    loss_per_fold = []
    corona_acc=[]
    corona_loss=[]
    pneumonia_acc=[]
    pneumonia_loss=[]
    normal_acc=[]
    normal_loss=[]
    #Merge Training and Test Data
    # x = np.concatenate((x_train, x_test), axis=0)
    # y = np.concatenate((y_train, y_test), axis=0)

    # Define the K-fold Cross Validator
    num_folds = 10
    # kfold = KFold(n_splits=num_folds, shuffle=True)
    # kfold = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=1)

    # K-fold Cross Validation model evaluation
    fold_no = 10
    for i in range(fold_no,11):

      f = open("/content/drive/My Drive/Corona_Image/E3_patient_wise/samll_model_custom_weight.txt", "a")

      densenet = DenseNet121(
        weights='/content/drive/My Drive/Corona_Image/chexnet_weights.h5',
        # weights = None ,
        include_top=True,
        input_shape=(224,224,3),classes=14
      )
      
      m = densenet.layers[-2].output
      # y=BatchNormalization(axis=1, momentum=0.99, epsilon=0.001, center=True, 
      #                               scale=True, beta_initializer='zeros', gamma_initializer='ones', 
      #                               moving_mean_initializer='zeros', moving_variance_initializer='ones',
      #                               beta_regularizer=None, gamma_regularizer=None, beta_constraint=None, gamma_constraint=None)(m)
      # z=Dropout(drop)(y)
      predictions = Dense(3, activation='softmax')(m)
      model = Model(inputs=densenet.inputs, outputs=predictions)

      adam = Adamax(lr=l, beta_1=0.9, beta_2=0.999)
      model.compile(loss='categorical_crossentropy',optimizer=adam,metrics=['accuracy'])
      reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5)

      es = EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=0, mode='auto', baseline=None, restore_best_weights=True)


      # Generate a print
      print('------------------------------------------------------------------------')
      print(f'Training for fold {fold_no} ...')
      
      class_weights = {0: 2, 1: 3, 2: 5}

      # Train data load
      train_data, val_data, train_label, val_label = load_train(fold_no)
      #class_weights = class_weight.compute_class_weight('balanced', np.unique(train_label), train_label)
      train_label = np_utils.to_categorical(train_label, 3)
      val_label = np_utils.to_categorical(val_label, 3)
      
      history = model.fit(train_data, train_label, batch_size=batch,
                          epochs=100,verbose=1, validation_data=(val_data,val_label), shuffle=True, class_weight=class_weights,
                            callbacks=[reduce_lr, es])
      
      # Generate generalization metrics
      x_test, y_test_new = load_test(fold_no)


      y_test = np_utils.to_categorical(y_test_new, 3)
      scores = model.evaluate(x_test, y_test, verbose=0)
      print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
      f.write(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
      acc_per_fold.append(scores[1] * 100)
      loss_per_fold.append(scores[0])
      
      corona_test = x_test[y_test_new == 2]
      corona = model.predict(corona_test)
      corona_test_label = y_test[y_test_new == 2]
      scores=model.evaluate(corona_test,corona_test_label)
      print(scores)
      corona_acc.append(scores[1] * 100)
      corona_loss.append(scores[0])
      f.write(f'> Fold {fold_no} - corona_loss: {scores[0]} - corona_acc: {scores[1] * 100}%')


      corona_test = x_test[y_test_new == 1]
      pneumonia = model.predict(corona_test)
      corona_test_label = y_test[y_test_new == 1]
      scores=model.evaluate(corona_test,corona_test_label)
      print(scores)
      pneumonia_acc.append(scores[1] * 100)
      pneumonia_loss.append(scores[0])
      f.write(f'> Fold {fold_no} - pneumonia_loss: {scores[0]} - pneumonia_acc: {scores[1] * 100}%')

      corona_test = x_test[y_test_new == 0]
      normal = model.predict(corona_test)
      corona_test_label = y_test[y_test_new == 0]
      scores=model.evaluate(corona_test,corona_test_label)
      print(scores)
      normal_acc.append(scores[1] * 100)
      normal_loss.append(scores[0])
      f.write(f'> Fold {fold_no} - normal_loss: {scores[0]} - normal_acc: {scores[1] * 100}%')
      

      corona_classes = np.unique([np.argmax(y, axis=None, out=None) for y in corona], return_counts=True)
      pneumonia_classes = np.unique([np.argmax(y, axis=None, out=None) for y in pneumonia], return_counts=True)
      normal_classes = np.unique([np.argmax(y, axis=None, out=None) for y in normal], return_counts=True)
      print(corona_classes)
      print(pneumonia_classes)
      print(normal_classes)
      f.write("corona_classes :"+str(corona_classes)+"\n")
      f.write("pneumonia_classes :"+str(pneumonia_classes)+"\n")
      f.write("normal_classes :"+str(normal_classes)+"\n")

      f.close()

      #model.save(f'/content/drive/My Drive/Corona_Image/E3_patient_wise/Models/{fold_no}.h5')

      # Increase fold number
      fold_no = fold_no + 1

      #del model,x_train,y_train,x_test,y_test,y_test_new,train_label

      # models.append(model)

    # == Provide average scores ==
    f = open("/content/drive/My Drive/Corona_Image/E3_patient_wise/samll_model_custom_weight.txt", "a")
    print('------------------------------------------------------------------------')
    print('Score per fold')
    for i in range(0, len(acc_per_fold)):
      print('------------------------------------------------------------------------')
      print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
      print(f'> Fold {i+1} - corona_loss: {corona_loss[i]} - corona_acc: {corona_acc[i]}%')
      print(f'> Fold {i+1} - pneumonia_loss: {pneumonia_loss[i]} - pneumonia_acc: {pneumonia_acc[i]}%')
      print(f'> Fold {i+1} - normal_loss: {normal_loss[i]} - normal_acc: {normal_acc[i]}%')
    print('------------------------------------------------------------------------')
    print('Average scores for all folds:')
    print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
    print(f'> Loss: {np.mean(loss_per_fold)}')

    print(f'> corona_acc: {np.mean(corona_acc)} (+- {np.std(corona_acc)})')
    print(f'> corona_loss: {np.mean(corona_loss)}')

    print(f'> pneumonia_acc: {np.mean(pneumonia_acc)} (+- {np.std(pneumonia_acc)})')
    print(f'> pneumonia_loss: {np.mean(pneumonia_loss)}')

    print(f'> normal_acc: {np.mean(normal_acc)} (+- {np.std(normal_acc)})')
    print(f'> normal_loss: {np.mean(normal_loss)}')
    print('------------------------------------------------------------------------')

    f.write('Average scores for all folds:')
    f.write(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
    f.write(f'> Loss: {np.mean(loss_per_fold)}')

    f.write(f'> corona_acc: {np.mean(corona_acc)} (+- {np.std(corona_acc)})')
    f.write(f'> corona_loss: {np.mean(corona_loss)}')

    f.write(f'> pneumonia_acc: {np.mean(pneumonia_acc)} (+- {np.std(pneumonia_acc)})')
    f.write(f'> pneumonia_loss: {np.mean(pneumonia_loss)}')

    f.write(f'> normal_acc: {np.mean(normal_acc)} (+- {np.std(normal_acc)})')
    f.write(f'> normal_loss: {np.mean(normal_loss)}')
    f.write('------------------------------------------------------------------------')
    f.close()

In [0]:
define_mode(.00001,16)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


------------------------------------------------------------------------
Training for fold 10 ...

Train on 22074 samples, validate on 2608 samples
Epoch 1/100
22074/22074 [==============================] - 246s 11ms/step - loss: 3.4923 - accuracy: 0.4634 - val_loss: 0.9432 - val_accuracy: 0.5054
Epoch 2/100
22074/22074 [==============================] - 219s 10ms/step - loss: 2.8074 - accuracy: 0.5193 - val_loss: 0.8060 - val_accuracy: 0.6472
Epoch 3/100
22074/22074 [==============================] - 220s 10ms/step - loss: 2.4905 - accuracy: 0.6240 - val_loss: 0.6958 - val_accuracy: 0.7063
Epoch 4/100
22074/22074 [==============================] - 220s 10ms/step - loss: 2.2398 - accuracy: 0.6677 - val_loss: 0.6016 - val_accuracy: 0.7389
Epoch 5/100
22074/22074 [==============================] - 221s 10ms/step - loss: 1.9849 - accuracy: 0.6923 - val_loss: 0.5160 - val_accuracy: 0.7588
Epoch 6/100
22074/2

In [0]:
define_mode(.00001,16)

------------------------------------------------------------------------
Training for fold 10 ...
Train on 22074 samples, validate on 2608 samples
Epoch 1/100
22074/22074 [==============================] - 245s 11ms/step - loss: 3.1829 - accuracy: 0.4575 - val_loss: 0.9322 - val_accuracy: 0.4850
Epoch 2/100
22074/22074 [==============================] - 224s 10ms/step - loss: 2.7684 - accuracy: 0.4563 - val_loss: 0.8244 - val_accuracy: 0.5077
Epoch 3/100
22074/22074 [==============================] - 225s 10ms/step - loss: 2.4752 - accuracy: 0.5125 - val_loss: 0.7129 - val_accuracy: 0.6204
Epoch 4/100
16976/22074 [======================>.......] - ETA: 49s - loss: 2.1928 - accuracy: 0.6190

KeyboardInterrupt: ignored